In [102]:
import numpy as np
import math
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.translate.metrics import alignment_error_rate
from nltk.metrics.scores import (precision, recall)

In [64]:
#DATA
file = open('./train.en', 'r')
train_en = file.read()
sentences_train_en = train_en.split("\n")

file = open('./train.hi', 'r')
train_hi = file.read()
sentences_train_hi = train_hi.split("\n")

file = open('./test.en', 'r')
test_en = file.read()
sentences_test_en = test_en.split("\n")

file = open('./test.hi', 'r')
test_hi = file.read()
sentences_test_hi = test_hi.split("\n")

file = open('./dev.en.txt', 'r')
dev_en = file.read()
sentences_dev_en = dev_en.split("\n")

file = open('./dev.hi', 'r')
dev_hi = file.read()
sentences_dev_hi = dev_hi.split("\n")

In [77]:
def is_converged(new, old, epoch):
    epsilone = 0.0000000001
    new = list(new.values())
    old = list(old.values())
    
    for i in range(len(old)):
        if math.fabs(new[i]-old[i]) > epsilone: 
            return False
        return True

In [75]:
def perform_EM(en_sentences, hi_sentences):
    
    translation_prob = defaultdict(float)
    translation_prob_prev = defaultdict(float)
    uni_ini = 0.00001
    
    epoch = 0
    
    while True:

        epoch += 1
        print("epoch num:", epoch,"\n")
        count = defaultdict(float)
        total = defaultdict(float)
        for index_sen, hin_sen in enumerate(hi_sentences):
            #compute normalization
            hin_sen_words = hin_sen.split(" ")
            s_total = defaultdict(float)
            for hin_word in hin_sen_words:
                s_total[hin_word] = 0
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                    if epoch == 1:
                        s_total[hin_word] += uni_ini
                        translation_prob[(hin_word, eng_word)] = uni_ini
                    else:
                        s_total[hin_word] += translation_prob[(hin_word, eng_word)]
            
            #collect counts
            for hin_word in hin_sen_words:
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                    if epoch == 1:
                        translation_prob[(hin_word, eng_word)] = uni_ini
                        count[(hin_word, eng_word)] += uni_ini/s_total[hin_word]
                        total[eng_word] += uni_ini/s_total[hin_word]
                    else:
                        count[(hin_word, eng_word)] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]
                        total[eng_word] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]                   

        #estimate probabilities
        for (hin_word, eng_word) in translation_prob.keys():
                translation_prob[(hin_word, eng_word)] = count[(hin_word, eng_word)]/total[eng_word]

        if is_converged(translation_prob, translation_prob_prev, epoch) == True:
            break
        
        translation_prob_prev = translation_prob
        
        
    return translation_prob


In [67]:
def train_model(sentences_train_en, sentences_train_hi):
    
    translation_prob = perform_EM(sentences_train_en, sentences_train_hi)
    return translation_prob


In [20]:
def test_model(dataset, tef):
#     tef = np.load('./models/IBMmodel1tef.npy')
    for sentence in dataset:
        translate_sentence(sentence, tef)


In [104]:
def translate_sentence(sentence, tef):
    
    ## take the best translation of an english word
    ## and append to hindi sentence
    print("s:",sentence)
    tokens = sentence.split(" ")
    for idx, token in enumerate(tokens):
        print(idx)
        max_score = -1
        for i in range(len(tef)):
#             print("search i:", i, token.lower(), list(tef.keys())[i][1].lower())
            if token.lower() == list(tef.keys())[i][1].lower():
                print("i", list(tef.items())[i])

    
#     max_sentence = ""
    
#     prob = get_translation_prob(end_sentence, tef)
    
#     if prob > max_score:
#         max_score = prob
#         max_sentence = poss_sentence

In [76]:
tef = train_model(sentences_train_en, sentences_train_hi)

epoch num: 1 

epoch num: 2 

s 2


In [ ]:
# np.save("./models/IBMmodel1tef_3", tef)

In [98]:
# print(list(tef)[:60])

In [ ]:
test_model(sentences_dev_en, tef)

s: but you will also be safe from eye diseases .
0


In [11]:
def find_recall_precision(predicted_translations):
    y_pred = list(predicted_translations)
    y_true = sentences_dev_hi
    recall = recall(y_true, y_pred, average=None)
    precision = precision(y_true, y_pred, average=None)
    return recall, precision;